In [1]:
import pandas as pd
import requests
import time
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Reads addresses from Excel
addresses_df = pd.read_excel("Sample addresses.xlsx", header=None)
addresses = addresses_df[0].tolist()

API_KEY = "Add your API Key here" # OpenCage API Key

In [2]:
def geocode_address(address):
    endpoint = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={API_KEY}"
    response = requests.get(endpoint)
    payload = response.json()
    if payload["results"]:
        location = payload["results"][0]["geometry"]
        return (location["lat"], location["lng"])
    return None

locations = [geocode_address(addr) for addr in addresses]

# Respect API rate limits
time.sleep(1)

# Creating distance matrix using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = (
        math.sin(delta_phi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c * 1000  # Convert to meters

def create_distance_matrix(locations):
    matrix = []
    for from_node in locations:
        row = []
        for to_node in locations:
            if from_node == to_node:
                row.append(0)
            else:
                row.append(int(haversine(from_node[0], from_node[1], to_node[0], to_node[1])))
        matrix.append(row)
    return matrix

distance_matrix = create_distance_matrix(locations)

# Creating data model
data = {
    "distance_matrix": distance_matrix,
    "num_vehicles": 4,
    "depot": 0
}

# Setting up routing model
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)
routing = pywrapcp.RoutingModel(manager)

# Creating and registering a transit callback
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Adding Distance constraint
dimension_name = "Distance"
routing.AddDimension(
    transit_callback_index,
    0,
    30000000,
    True,
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Defining search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

In [3]:
# Solving
solution = routing.SolveWithParameters(search_parameters)

# Printing solution
def print_solution(data, manager, routing, solution):
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route.append(manager.IndexToNode(index))
        print(f"Route for vehicle {vehicle_id}: {route}")
        print(f"Distance for vehicle {vehicle_id}: {route_distance / 1000:.2f} km")

if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found.")

Route for vehicle 0: [0, 12, 8, 0]
Distance for vehicle 0: 8257.56 km
Route for vehicle 1: [0, 10, 0]
Distance for vehicle 1: 4568.94 km
Route for vehicle 2: [0, 7, 6, 3, 0]
Distance for vehicle 2: 8173.40 km
Route for vehicle 3: [0, 9, 5, 11, 1, 4, 2, 0]
Distance for vehicle 3: 8256.56 km
